In [49]:
import trodes.read_exported as tr
import os
import json
import numpy as np

In [ ]:
# insert path to folder with videoTimestamps files and merged.time folders for each recording
path = r"C:\Users\megha\UFL Dropbox\Meghan Cum\Padilla-Coreano Lab\2024\Cum_SocialMemEphys_pilot2\Object_control_ephys\timestamps"


In [ ]:
# use this cell to extract first timestamp from each recording 
# find a similar pattern between merged.time folder names and videotimestamps file names
# in this case the date is the smae across both file types

# side note: first timestamp per recording in a recording session are the same 
# ei if you recorded from 4 headstages at the same time (all in the same rec folder)
# each one will have the same first timestamp 

first_timestamp_dict = {}
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".dat"):
            ts_file = os.path.join(root, file)
            ts_dict = tr.read_trodes_extracted_data_file(ts_file)
            first_timestamp = ts_dict['first_timestamp']
            file_parts = file.split('_')
            file_pattern = file_parts[0] + '_' + file_parts[1]
            first_timestamp_dict[file_pattern] = int(first_timestamp)

print(first_timestamp_dict)

{'20250618_113931': 1959619, '20250618_123431': 1665815, '20250618_133349': 2437744, '20250618_143034': 1243125}


In [ ]:
# this cell will create a dictionary with three items under each merged.rec file name
# stream_indexed_array: camera module timestamps in seconds where the 0th second is at stream
# play_indexed_array: camera module timestamps in seconds where the 0th second is at play (when you clicked play in trodes)
# first_timestamp: the timestamp in 20KHz of when you clicked play, where the 0th timestamp is at stream

play_indexed_dict = {}
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".videoTimeStamps"):
            play_indexed_dict[file] = {}
            videotsfile = os.path.join(root, file)
            videotsarray = tr.readCameraModuleTimeStamps(videotsfile)
            date = file.split('_')[0] + '_' + file.split('_')[1]
            first_ts = first_timestamp_dict[date]
            videotsarray_play_indexed = videotsarray - first_ts/20000
            if videotsarray_play_indexed[0] < 0:
                if abs(videotsarray_play_indexed[0]) < .001:
                    videotsarray_play_indexed[0] = 0
                else:
                    print('negative first timestamp: ', videotsarray_play_indexed[0])
            play_indexed_dict[file]['play_indexed_array'] = videotsarray_play_indexed
            play_indexed_dict[file]['first_timestamp'] = first_ts
            play_indexed_dict[file]['stream_indexed_array'] =  videotsarray 




In [ ]:
# this function saves the dictionary to a json to be used later when creating an event dict
# that is accurately aligned to the neural data timestamps

def convert_numpy_to_list(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_to_list(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_to_list(item) for item in obj]
    else:
        return obj

# Convert the dictionary for JSON serialization
json_compatible_dict = convert_numpy_to_list(play_indexed_dict)

# Save to JSON file
output_filename = 'eli_object_control_play_indexed_timestamps.json'
with open(output_filename, 'w') as f:
    json.dump(json_compatible_dict, f, indent=2)